## Run the block to access google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, pickle, datetime, warnings, sys, os
warnings.filterwarnings('ignore')
# data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Loan_participants_data.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Assign the path to the directory containing the files.

In [27]:
parent_path = '/content/drive/MyDrive/Colab Notebooks/project'

In [28]:
empty_records = lambda : records.drop(range(len(records)), axis=0).to_csv(os.path.join(parent_path, 'record.csv'), index=False)
sys.path.append(parent_path)
from risk import *
pr = prediction(os.path.join(parent_path,'xgb_model_tuned2.pickle'),
                os.path.join(parent_path,'scaler.pickle'))

## Fill the following fields and execute to get prediction.

In [26]:
records = pd.read_csv(os.path.join(parent_path,'record.csv'))
records.id = records.id.astype('string')

#@title Loan eligibility check { form-width: "30%" }
Name = 'customer14' #@param {type:"string"}
ID = '3533678' #@param {type:"string"}
Mobile_no = '2326653543' #@param {type:"string"}
Current_loan_amount = 75526 #@param {type:"number"}
Long_term_loan = False #@param {type:"boolean"}
Credit_score = 716 #@param {type:"slider", min:300, max:900, step:1}
Household_income = 138323.1 #@param {type:"number"}
Years_in_currenct_job = '\u003C 1 year' #@param ['10+ years', '8 years', '6 years', '3 years', '< 1 year', '4 years', '5 years', '2 years', '9 years', '1 year', '7 years'] {type:"string"}
Home_mortgaged = False #@param {type:"boolean"}
Loan_type = "Personal Loan" #@param ['Personal Loan', 'Home Loan', 'Business Loan', 'shopping loan', 'other', 'Vehicle Loan', 'Education Loan', 'wedding loan'] {type:"string"}
Monthly_debt = 1477.06 #@param {type:"number"}
Years_of_credit_history = 11 #@param {type:"slider", min:0, max:70, step:0.1}
# Employee_index = 'other' #@param ['ex_employed', 'filial', 'not_employee', 'other'] {type:"string"}
# Country = 'ES' #@param ['other', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BO', 'BR', 'BY', 'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CR', 'CU', 'CZ', 'DE', 'DK', 'DO', 'EC', 'ES', 'ET', 'FI', 'FR', 'GB', 'GT', 'GW', 'IE', 'IL', 'IT', 'JP', 'KE', 'KW', 'MA', 'MX', 'MZ', 'NL', 'OM', 'PE', 'PL', 'PT', 'PY', 'RO', 'RU', 'SA', 'SE', 'SG', 'SL', 'TG', 'TH', 'UA', 'US', 'UY', 'VE', 'ZA'] {type:"string"}
# Gender = 'M' #@param ['F', 'M'] {type:"string"}
# Age = 25 #@param {type:"number"}
# Born_in_Spain = True #@param {type:'boolean"}
# Customer_holding_date = '2023-10-18' #@param {type:"date"}
# Registered_in_last_6_months = False #@param {type:'boolean"}
# Customer_seniority_in_months = 5 #@param {type:"number"}
# Customer_rank = 99 #@param ['Primary, 'Primary customer during the month but not at the end of the month'] {type:"string"}
# Customer_rank_lastday = '2023-10-18' #@param {type:"date"}
# Customer_type = 'primary' #@param ['primary', 'former_primary', 'former_co-owner', 'potential', 'other'] {type:"string"}
# Relation_type = 'former' #@param ['former', 'inactive', 'potential', 'other'] {type:"string"}
# Channel = 'KFC' #@param ['KFA', 'KFC', 'KHE', 'KHQ', 'other'] {type:"string"}
# Activity_index = 'Active' #@param ['Active','Inactive'] {type:"string"}
# Customer_segment = 'individual' #@param ['individual', 'other', 'vip'] {type:"string"}

risk_fields = {'home_mortgaged':Home_mortgaged,
 'years_of_credit_history':Years_of_credit_history,
 'years_in_current_job':Years_in_currenct_job,
 'monthly_debt':Monthly_debt,
 'credit_score':Credit_score,
 'long_term':Long_term_loan,
 'loan_type':Loan_type,
 'household_income':Household_income,
 'current_loan_amount':Current_loan_amount}


# recommender_fields = {
#     'emp_index': Employee_index,
#     'country': Country,
#     'gender': Gender,
#     'age': Age,
#     'birth_index': Born_in_Spain,
#     'cust_holding_date': Customer_holding_date,
#     'new_cust_index': Registered_in_last_6_months,
#     'cust_seniority': Customer_seniority_in_months,
#     'cust_rank': Customer_rank,
#     'cust_rank_lastday': Customer_rank_lastday,
#     'cust_type': Customer_type,
#     'relation_type': Relation_type,
#     'channel': Channel,
#     'activity_index': Activity_index,
#     'household_income': Household_income,
#     'cust_segment': Customer_segment
# }
# print(int(ID) in records.id.unique())
if str(ID) not in [str(i) for i in records.id.unique()]:

  pred = pr.risk_predict(**risk_fields)
  now = datetime.datetime.now()
  date = now.strftime("%Y-%m-%d")
  time = now.strftime("%H:%M:%S")

  if pred=='Will pay':
    recoms = []
    ln_tps = ['Personal Loan', 'Home Loan', 'Business Loan', 'shopping loan', 'other', 'Vehicle Loan', 'Education Loan', 'wedding loan']
    for tp in ln_tps:
      cstm_flds = risk_fields.copy()
      cstm_flds['loan_type'] = tp
      if pr.risk_predict(**cstm_flds) == 'Will pay':
        recoms.append(tp)
    # recommendation = pr.recommend(**recommender_fields)
    print(pred,"\nYou are eligible of getting the following loans:\n"+'\n'.join(recoms))
  else:
    recoms = []
    print(pred)

  records.loc[len(records)] = [date, time, Name,
                              ID, Mobile_no]+list(risk_fields.values()) + \
                              [pred, ', '.join(recoms)]
  records.to_csv(os.path.join(parent_path,'record.csv'), index=False)
else:
  dup = records[records.id==str(ID)]
  last_pred = dup[pd.to_datetime(dup.date +' '+ dup.time) == pd.to_datetime(dup.date +' '+ dup.time).max()]['risk_prediction'].values[0]
  last_pred_date = dup[pd.to_datetime(dup.date +' '+ dup.time) == pd.to_datetime(dup.date +' '+ dup.time).max()]['date'].values[0]
  last_pred_time = dup[pd.to_datetime(dup.date +' '+ dup.time) == pd.to_datetime(dup.date +' '+ dup.time).max()]['time'].values[0]
  inp = input(f'Data already present for this customer in index nos. {dup.index.tolist()}\
 with last prediction as "{last_pred}" on {last_pred_date} at {last_pred_time}. Do you want to perform another instance of analysis? Y/N: ')
  if inp.upper()=='Y':
    pred = pr.risk_predict(**risk_fields)
    now = datetime.datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    if pred=='Will pay':
      recoms = []
      ln_tps = ['Personal Loan', 'Home Loan', 'Business Loan', 'shopping loan', 'other', 'Vehicle Loan', 'Education Loan', 'wedding loan']
      for tp in ln_tps:
        cstm_flds = risk_fields.copy()
        cstm_flds['loan_type'] = tp
        if pr.risk_predict(**cstm_flds) == 'Will pay':
          recoms.append(tp)
      # recommendation = pr.recommend(**recommender_fields)
      print(pred,"\nYou are eligible of getting the following loans:\n"+'\n'.join(recoms))
    else:
      recoms = []
      print(pred)

    records.loc[len(records)] = [date, time, Name,
                                ID, Mobile_no]+list(risk_fields.values()) + \
                                [pred, ', '.join(recoms)]

  records.to_csv(os.path.join(parent_path,'record.csv'), index=False)
output = records[records.id==str(ID)]

Data already present for this customer in index nos. [20] with last prediction as "Will pay" on 2023-10-29 at 10:19:32. Do you want to perform another instance of analysis? Y/N: n


## Execute the following to get details pertaining to the specific queried customer

In [20]:
output

,date,time,name,id,mobile_no,home_mortgaged,years_of_credit_history,years_in_current_job,monthly_debt,credit_score,long_term,loan_type,household_income,current_loan_amount,risk_prediction,recommendations
19,2023-10-29,09:19:22,customer13,347613678,232666389,False,13.6,< 1 year,12320.17,722,False,Personal Loan,56029.02,326106.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."


## Execute to check the customers in records, their details, predictions and recommendations.

In [29]:
records

,date,time,name,id,mobile_no,home_mortgaged,years_of_credit_history,years_in_current_job,monthly_debt,credit_score,long_term,loan_type,household_income,current_loan_amount,risk_prediction,recommendations
0,2023-10-26,11:14:14,customer1,1234,23231234,True,10.7,< 1 year,1712.72,743,True,Business Loan,8.000000e+00,481294.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
1,2023-10-26,11:15:04,customer2,12345,23231234,True,18.5,6 years,155712.72,672,True,Business Loan,8.000000e+00,48129409.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
2,2023-10-26,11:16:03,customer3,12345,23231234,False,18.5,6 years,155712.72,580,False,Education Loan,1.366500e+04,129409.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
3,2023-10-26,11:19:32,customer4,123456,23231234,False,16.6,10+ years,18348.68,714,True,Personal Loan,1.100435e+05,285406.0,Will default,NaN
4,2023-10-26,11:21:56,customer5,1234567,23231234,False,14.0,8 years,21110.90,736,False,Personal Loan,1.100435e+05,132330.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
5,2023-10-26,11:22:56,customer6,34567,23231234,False,14.0,8 years,21145710.90,673,False,Personal Loan,1.100333e+08,132330.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
6,2023-10-26,11:23:17,customer6,345678,23231234,False,14.0,8 years,21145710.90,654,False,Personal Loan,1.100333e+08,13233088.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
7,2023-10-26,11:23:55,customer6,3478,23231234,True,14.0,8 years,21145710.90,654,False,Personal Loan,1.100333e+08,13233088.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
8,2023-10-26,11:25:02,customer6,3478,23231234,True,14.0,1 year,21145710.90,654,False,Personal Loan,1.100333e+08,13233088.0,Will pay,"Personal Loan, Home Loan, Business Loan, shopp..."
9,2023-10-26,11:26:37,customer7,34789,23231234,True,30.5,10+ years,21739.80,695,True,Personal Loan,6.699102e+04,786170.0,Will default,NaN


## Execute the following to delete all customer records.

In [ ]:
# empty_records()

# ------------------------------------------------------------END------------------------------------------------------------------

## Plauground

In [23]:
from tqdm.notebook import tqdm

In [24]:
fnd = []
for row in tqdm(data.dropna().sample(n=200).iterrows()):
  Current_loan_amount = float(row[1]['Current Loan Amount'])
  Long_term_loan = False if row[1]['Term']=='Short Term' else True
  Credit_score = int(row[1]['Credit Score'])
  Household_income = float(row[1]['household_income'])
  Years_in_currenct_job = row[1]['Years in current job']
  Home_mortgaged = False if row[1]['Home Ownership']!='Home Mortgage' else True
  Loan_type = row[1]['Loan_type']
  Monthly_debt = float(row[1]['Monthly Debt'])
  Years_of_credit_history = float(row[1]['Years of Credit History'])

  risk_fields = {'home_mortgaged':Home_mortgaged,
  'years_of_credit_history':Years_of_credit_history,
  'years_in_current_job':Years_in_currenct_job,
  'monthly_debt':Monthly_debt,
  'credit_score':Credit_score,
  'long_term':Long_term_loan,
  'loan_type':Loan_type,
  'household_income':Household_income,
  'current_loan_amount':Current_loan_amount}

  pred = pr.risk_predict(**risk_fields)

  if pred=='Will pay':
    recoms = []
    ln_tps = ['Personal Loan', 'Home Loan', 'Business Loan', 'shopping loan', 'other', 'Vehicle Loan', 'Education Loan', 'wedding loan']
    for tp in ln_tps:
      cstm_flds = risk_fields.copy()
      cstm_flds['loan_type'] = tp
      if pr.risk_predict(**cstm_flds) == 'Will pay':
        recoms.append(tp)

    if len(recoms) < 8:
      # print('Found')
      fnd.append(risk_fields)
  # except:
  #   continue

0it [00:00, ?it/s]

In [25]:
len(fnd)

18

In [26]:
fnd

[{'home_mortgaged': True,
  'years_of_credit_history': 24.6,
  'years_in_current_job': '10+ years',
  'monthly_debt': 32358.9,
  'credit_score': 734,
  'long_term': False,
  'loan_type': 'Personal Loan',
  'household_income': 110143.41,
  'current_loan_amount': 743424.0},
 {'home_mortgaged': True,
  'years_of_credit_history': 28.3,
  'years_in_current_job': '10+ years',
  'monthly_debt': 10633.54,
  'credit_score': 697,
  'long_term': False,
  'loan_type': 'Personal Loan',
  'household_income': 229147.05,
  'current_loan_amount': 259798.0},
 {'home_mortgaged': True,
  'years_of_credit_history': 17.6,
  'years_in_current_job': '10+ years',
  'monthly_debt': 36262.45,
  'credit_score': 689,
  'long_term': True,
  'loan_type': 'Personal Loan',
  'household_income': 132294.69,
  'current_loan_amount': 285296.0},
 {'home_mortgaged': False,
  'years_of_credit_history': 17.5,
  'years_in_current_job': '2 years',
  'monthly_debt': 5781.7,
  'credit_score': 693,
  'long_term': False,
  'loan_ty